This script is to add data general. 
Import all required tools

In [1]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df

%matplotlib inline

mp = ixmp.Platform('local')

<IPython.core.display.Javascript object>

This part is just to generate a dummy scenario as basis for adding technology parameters. Need to be removed once integrated

In [2]:
model = "Westeros Electrified"

base = message_ix.Scenario(mp, model=model, scenario="baseline")
scen = base.clone(
    model,
    "add_tech",
    "introducing the add_tech feature on MESSAGEix",
    keep_solution=False,
)
scen.check_out()

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")
country = "Westeros"


In [215]:
# Read DACCS technology data
with open('tech_data.yaml','r') as stream:
    tech_data = yaml.safe_load(stream)

# Set up dictionary of parameter indices list
par_idx = {}
data = {}

# Create dicitonary of parameter indices and data
for tech in set(tech_data):
    # add vintage and active years and update tech_data for each tech
    years_vtg_act = scen.vintage_and_active_years()
    years_vtg_act = years_vtg_act[years_vtg_act['year_vtg'] >= tech_data[tech]['year_init']]
    tech_data[tech]['year_vtg']= years_vtg_act['year_vtg'].to_list()
    tech_data[tech]['year_act']= years_vtg_act['year_act'].to_list()

    # collect parameter indices and update data
    par_idx.update({tech: {par: {idx: [] for idx in list(MESSAGE_ITEMS[par].get('idx_names'))} 
                           for par in set(tech_data[tech]) - set(['year_init','year_vtg','year_act'])
                          }})
    
    data.update({tech: {par: [] for par in list(par_idx[tech].keys())}})
    

In [197]:
#tech_data

In [216]:
# If those are not provided, then this block of code is needed to retrieve them from the data input

regions = []
emissions = []
times = []
modes = []
commodities = []
levels = []
relations = []

set_elements_dict = {
    'node_loc'   :{'data': regions,     'name':'node'},
    'emission'   :{'data': emissions,   'name':'emission'},
    'mode'       :{'data': modes,       'name':'mode'},
    'time'       :{'data': times,       'name':'time'},
    'commodity'  :{'data': commodities, 'name':'commodity'},
    'level'      :{'data': levels,      'name':'level'},
    'time_origin':{'data': times,       'name':'time'},
    'time_dest'  :{'data': times,       'name':'time'},
    'relation'   :{'data': relations,   'name':'relation'},
    'node_rel'   :{'data': regions,     'name':'node'},
}   

# investment cost


In [265]:
# DONOT DELETE
tec = 'DACCS'
par = 'inv_cost'

kwargs = {}
if all(idx in par_idx[tech][par] for idx in ['year_vtg','year_act']):
    kwargs = {
        'year_vtg': tech_data[tech]['year_vtg'],
        'year_act': tech_data[tech]['year_act']
    }
elif 'year_vtg' in par_idx[tech][par]:
    kwargs = {
        'year_vtg': sorted(set(tech_data[tech]['year_vtg']))
    }
else:
    kwargs = {
        'year_act': sorted(set(tech_data[tech]['year_act']))
    }
    # if 'year_rel' is present, the values are assumed from 'year_act' values
    if 'year_rel' in par_idx[tech][par]:
        kwargs.update(
            {'year_rel': sorted(set(tech_data[tech]['year_act']))}
        )

df = make_df(par,
             technology=tec,
             value=tech_data[tec][par]['value'],
             unit=tech_data[tec][par]['unit'],
             **kwargs)    

# create empty dataframe
idx_exp = [e for e in par_idx[tec][par] 
           if e not in ['technology','year_vtg','year_act',
                        'node_origin','node_dest',
                        'time_origin','time_dest',
                       ]]

for idx in idx_exp:
    default = (
        {e:1 for e in list(scen.set(set_elements_dict[idx]['name']))[1:]} 
        if idx in ['node_loc','mode']
        else {e:1 for e in list(scen.set(set_elements_dict[idx]['name']))}
    )
    
    listidx = list(tech_data[tec][par].get(idx,default))
    listdfidx = []
    for e in listidx:
        df1 = df.copy()
        df1[idx] = [e] * len(df)
        listdfidx.append(df1)
    df = pd.concat(listdfidx, ignore_index=True)

# assigning values for node and time related indices
for idx in df.columns:
    if idx in ['node_origin','node_dest']:
        df[idx] = df['node_loc']
    if idx in ['time_origin','time_dest']:
        df[idx] = df['time']

# Calculate value row-by-row multiplier
mult = []
for i in range(len(df)):
    # node_loc factor
    _node_loc = (tech_data[tec]
                  .get(par,{})
                  .get('node_loc',{})
                  .get(df.get('node_loc',{}).get(2),1)
                 )

    # year_vtg factor
    # _year_vtg = (1+rate)**delta_years
    _year_vtg = (((1+tech_data[tec]
                   .get(par,{})
                   .get('year_vtg',{})
                   .get('rate',0))
                  **(df['year_vtg'][i] - tech_data[tech]['year_init']))
                  if 'year_vtg' in df.columns else 1
                )

    # year_act factor
    # _year_act = (1+rate)**(year_act-year_vtg) if both years present
    # _year_act = (1+rate)**(year_act-first_active_year) if no year_vtg
    _year_act = (((1+tech_data[tec]
                    .get(par,{})
                    .get('year_act',{})
                    .get('rate',0))
                   **(df['year_act'][i]
                     -(df['year_vtg'][i] 
                       if 'year_vtg' in df.columns 
                       else tech_data[tech]['year_init'])))
                  if 'year_act' in df.columns else 1
                )

    # get mode multiplier from model_data
    _mode = (tech_data[tec]
             .get(par,{})
             .get('mode',{})
             .get(df.get('mode',{}).get(i),1)
            )

    mult.append(
            np.prod([_node_loc, _year_vtg, _year_act, _mode,])
        )

# index adjusted df
value = df['value'] * mult
value = [round(e,3) for e in value]
df['value'] = value

df

,node_loc,technology,year_vtg,value,unit
0,Westeros,DACCS,700,2500.000,$/kW
1,Westeros,DACCS,710,1496.842,$/kW
2,Westeros,DACCS,720,896.215,$/kW


In [262]:
mult

[1.0, 0.5987369392383787, 0.3584859224085419]

In [253]:
tech_data

{'DACCS': {'year_init': 700,
  'inv_cost': {'unit': '$/kW',
   'value': 2500,
   'node_loc': {'Westeros': 1},
   'year_vtg': {'rate': -0.05}},
  'fix_cost': {'unit': '$/kWa',
   'value': 10,
   'node_loc': {'Westeros': 1},
   'year_vtg': {'rate': -0.05},
   'year_act': {'rate': 0.03}},
  'var_cost': {'unit': '$/kWa',
   'value': 1,
   'node_loc': {'Westeros': 1},
   'year_vtg': {'rate': -0.05},
   'year_act': {'rate': 0.03},
   'mode': {'standard': 1},
   'time': {'year': 1}},
  'emission_factor': {'unit': 'tCO2/kWa',
   'value': 1,
   'mode': {'standard': 1},
   'emission': {'CO2': -20, 'CH4': 0},
   'node_loc': {'Westeros': 1, 'Easteros': 1}},
  'input': {'unit': '-',
   'value': 1,
   'node_loc': {'Westeros': 1},
   'commodity': {'electricity': 1},
   'level': {'final': 1}},
  'output': {'unit': '-',
   'value': 0,
   'node_loc': {'Westeros': 1},
   'commodity': {'electricity': 0},
   'level': {'final': 0}},
  'capacity_factor': {'unit': '-', 'value': 1, 'node_loc': {'Westeros': 1}}

# Below is old script

In [5]:
# Read DACCS technology data
with open('tech_data - origin.yaml','r') as stream:
    tech_data = yaml.safe_load(stream)

# Set up dictionary of parameter indices list
par_idx = {}
data = {}

# Create dicitonary of parameter indices and data
for tech in set(tech_data) - set(['model_data']):
    par_idx.update({tech: {par: {idx: [] for idx in list(MESSAGE_ITEMS[par].get('idx_names'))} 
                           for par in set(tech_data[tech])}})
    data.update({tech: {par: [] for par in list(par_idx[tech].keys())}})


# Get scenario years related parameters    
first_active_year = tech_data['model_data'].get('first_active_year')
years_vtg_act = scen.vintage_and_active_years()
years_vtg_act = years_vtg_act[years_vtg_act['year_vtg'] >= first_active_year]


In [6]:
# If those are not provided, then this block of code is needed to retrieve them from the data input

regions = []
emissions = []
times = []
modes = []
commodities = []
levels = []
relations = []

set_elements_dict = {
    'node_loc'   :{'data': regions,     'name':'node'},
    'emission'   :{'data': emissions,   'name':'emission'},
    'mode'       :{'data': modes,       'name':'mode'},
    'time'       :{'data': times,       'name':'time'},
    'commodity'  :{'data': commodities, 'name':'commodity'},
    'level'      :{'data': levels,      'name':'level'},
    'time_origin':{'data': times,       'name':'time'},
    'time_dest'  :{'data': times,       'name':'time'},
    'relation'   :{'data': relations,   'name':'relation'},
    'node_rel'   :{'data': regions,     'name':'node'},
}   

for tec in par_idx.keys():
    for par in par_idx[tec]:
        for idx in set_elements_dict.keys():
            if idx in par_idx[tec][par].keys():
                if len(set_elements_dict[idx]['data']) != 0:
                    par_idx[tec][par][idx] = set_elements_dict[idx]['data']
                elif len(set_elements_dict[idx]['data']) == 0:
                    if tech_data['model_data'].get(tec,{}).get(par,{}).get(idx,{}) != {}:
                        par_idx[tec][par][idx] = list(tech_data['model_data'][tec][par][idx].keys())
                    else:
                        if idx in ['node_loc','mode']: # to not including 'World' and 'all' mode
                            par_idx[tec][par][idx] = list(scen.set(set_elements_dict[idx]['name']))[1:]
                        else:
                            par_idx[tec][par][idx] = list(scen.set(set_elements_dict[idx]['name']))


In [7]:
par_idx['DACCS']

{'abs_cost_activity_soft_up': {'node_loc': ['Westeros'],
  'technology': [],
  'year_act': [],
  'time': ['year']},
 'construction_time': {'node_loc': ['Westeros'],
  'technology': [],
  'year_vtg': []},
 'level_cost_activity_soft_up': {'node_loc': ['Westeros'],
  'technology': [],
  'year_act': [],
  'time': ['year']},
 'emission_factor': {'node_loc': ['Westeros'],
  'technology': [],
  'year_vtg': [],
  'year_act': [],
  'mode': ['standard'],
  'emission': ['CO2', 'CH4']},
 'input': {'node_loc': ['Westeros'],
  'technology': [],
  'year_vtg': [],
  'year_act': [],
  'mode': ['standard'],
  'node_origin': [],
  'commodity': ['electricity'],
  'level': ['final'],
  'time': ['year'],
  'time_origin': ['year']},
 'level_cost_activity_soft_lo': {'node_loc': ['Westeros'],
  'technology': [],
  'year_act': [],
  'time': ['year']},
 'output': {'node_loc': ['Westeros'],
  'technology': [],
  'year_vtg': [],
  'year_act': [],
  'mode': ['standard'],
  'node_dest': [],
  'commodity': ['electric

Creating basic dataframe to be filled in later

In [8]:
# Create basic DataFrame and expand according to model_data input
count = 0
for tech, par_dict in tech_data.items():
    if tech != 'model_data':
        for par, par_data in par_dict.items():
            if not isinstance(par_data, Mapping):
                par_data = {'value': par_data, 'unit': '-'}
            # identify parameters by year dimension
            # then add the year data as kwargs as input for basic dataframe
            if all(e in par_idx[tech][par] for e in ['year_vtg','year_act']):
                kwargs = {'year_vtg': years_vtg_act['year_vtg'],
                          'year_act': years_vtg_act['year_act']}
            elif 'year_vtg' in par_idx[tech][par]:
                kwargs = {'year_vtg': sorted(set(years_vtg_act['year_vtg']))}
            else:
                kwargs = {'year_act': sorted(set(years_vtg_act['year_act']))}
                # if 'year_rel' is present, the values are assumed from 'year_act' values
                if 'year_rel' in par_idx[tech][par]:
                    kwargs.update({'year_rel': sorted(set(years_vtg_act['year_act']))})
                                
            # create parameter's basic dataframe and 
            # add it to the data parameter list
            data[tech][par].append(
                make_df(
                    par,
                    technology=tech,
                    value=par_data['value'],
                    unit=par_data['unit'],
                    **kwargs,
                ))

            # duplicate the basic data using the length of each set
            # as the duplication factor
            for s in set_elements_dict.keys():
                if s in par_idx[tech][par]:# and s != 'node_loc':
                    elist = par_idx[tech][par][s]
                    data[tech][par] = data[tech][par]*len(elist)
                    for e in range(len(elist)):
                        kwarg = {s:elist[e]}
                        data[tech][par][e] = data[tech][par][e].assign(**kwarg)
                data[tech][par] = [pd.concat(data[tech][par]).reset_index(drop=True)]

data = {t: {k: pd.concat(v).reset_index(drop=True) 
            for k, v in data[t].items()} for t in data.keys()}
# data

In [9]:
data['DACCS']['inv_cost']

,node_loc,technology,year_vtg,value,unit
0,Westeros,DACCS,700,2500,$/kW
1,Westeros,DACCS,710,2500,$/kW
2,Westeros,DACCS,720,2500,$/kW


In [ ]:
# Expanded DataFrame
data_expand ={tech: {par: [] for par in data[tech].keys()} for tech in data.keys()}

for tech, par_data in tech_data['model_data'].items():
    if tech != 'first_active_year':
        for par in data[tech].keys():
            multiplier = []
            for i in range(len(data[tech][par])):
                # Calculate multipliers for each element in a dimensional array.
                # For each element, this function searches for corresponding factors
                # in the model-specific data (model_data).
                # If no factors are found, the multiplier is set to 1.
                # If factors are found, the function uses the factor that matches
                # the corresponding element in the data[par] row.

                # get regional multiplier from model_data
                #m_reg = par_data.get(par,{}).get('node_loc',{}).get(reg,1)
                m_node_loc = (par_data.get(par,{}).get('node_loc',{})
                              .get(data[tech][par].get('node_loc',{}).get(i),1))
                
                m_node_rel = (par_data.get(par,{}).get('node_rel',{})
                              .get(data[tech][par].get('node_rel',{}).get(i),1))

                # get year_vtg escalation rate from model_data
                # then calculate year_vtg multiplier
                # m_year_vtg = (1+rate)**delta_years
                m_year_vtg = (((1+par_data.get(par,{}).get('year_vtg',{}).get('rate',0)) 
                              **(data[tech][par]['year_vtg'][i]-first_active_year)) 
                              if 'year_vtg' in data[tech][par].columns else 1)

                # same as m_year_vtg
                # m_year_act = (1+rate)**(year_act-year_vtg) if both years present
                # m_year_act = (1+rate)**(year_act-first_active_year) if no year_vtg
                m_year_act = (((1+par_data.get(par,{}).get('year_act',{}).get('rate',0))
                              **(data[tech][par].get('year_act',{})
                                 .get(i,0)
                                 -(data[tech][par]['year_vtg'][i] 
                                   if 'year_vtg' in data[tech][par].columns else first_active_year)))
                             if 'year_act' in data[tech][par].columns else 1)

                m_year_rel = (((1+par_data.get(par,{}).get('year_rel',{}).get('rate',0))
                              **(data[tech][par].get('year_rel',{})
                                 .get(i,0)
                                 -(data[tech][par]['year_vtg'][i] 
                                   if 'year_vtg' in data[tech][par].columns else
                                   (data[tech][par]['year_act'][i] if 'year_act' in data[tech][par].columns 
                                    else first_active_year))))
                             if 'year_rel' in data[tech][par].columns else 1)

                
                # get mode multiplier from model_data
                m_mode = (par_data.get(par,{}).get('mode',{})
                          .get(data[tech][par].get('mode',{}).get(i),1))

                # get emission multiplier from model_data
                m_emission = (par_data.get(par,{}).get('emission',{})
                              .get(data[tech][par].get('emission',{}).get(i),1))

                multiplier.append(
                        np.prod([m_node_loc, m_node_rel, 
                                 m_year_vtg, m_year_act, m_year_act, 
                                 m_mode, m_emission,])
                    )

            value = data[tech][par]['value']*multiplier

            # node_origin and node_dest are assumed to be always the same of the node
            kwargs = {}
            if 'node_origin' in data[tech][par].columns:
                kwargs.update({'node_origin': data[tech][par]['node_loc'][i]})
            if 'node_dest' in data[tech][par].columns:
                kwargs.update({'node_dest': data[tech][par]['node_loc'][i]})

            # assigning data expansion
            data_expand[tech][par].append(
                data[tech][par].assign(value=value, **kwargs)
               )

In [ ]:
data_expand['DACCS']['inv_cost'][0]

In [ ]:
all_params = []
for t in data_expand.keys():
    for par in data_expand[t].keys():
        if par not in all_params:
            all_params.append(par)

data_to_scenario = {par: [] for par in all_params}

for k,v in data_expand.items():
    for k2,v2 in v.items():
        data_to_scenario[k2].append(v2[0])

data_expand = {k: pd.concat(v) for k, v in data_to_scenario.items()}        

In [ ]:
data_expand['inv_cost']

In [ ]:
with pd.ExcelWriter('printed_data.xlsx', engine='xlsxwriter', mode='w') as writer:
    for sheet_name, sheet_data in data_expand.items():
        sheet_data.to_excel(writer, sheet_name=sheet_name, index=False)

In [266]:
mp.close_db()